In [29]:
import pyarrow.parquet as pq
import geopandas as gpd
import numpy as np
import osmnx as ox
import h3
import folium
from pint import UnitRegistry
from copy import deepcopy
from uuid import uuid5, NAMESPACE_DNS
from shapely.geometry import LineString, Point, Polygon
import pyarrow as pa
import geoarrow.pyarrow as ga
import geoarrow.rust.io as gio
import json
import networkx as nx

### Prepare data to efficiently compute courier routes at runtime

We want to have a realistic simuation and provide realisitic looking data. Especially when plotting movement data it, sticking to roads / pathways make visulaizations much more compelling.
The baseic strategy is as follows.

1. select a sufficiently large region around our kitchen sites.
2. Load data from OSM and normalive node / edge information
3. Store as geoparquet so we can efficiently load it

Within the rust kernel, we can further prepare ourselves by creating a graph representation optimized for shortest path search.
This allows us to compute shortest path on every iteration.

In [ ]:
def process_nodes(location: str, G: nx.MultiDiGraph):
    node_ids = []
    node_coords = []
    node_props = []

    for id, meta in list(G.nodes(data=True)):
        properties = deepcopy(meta)
        node_coords.append(Point(properties["x"], properties["y"]))
        del properties["x"]
        del properties["y"]

        node_ids.append(uuid5(NAMESPACE_DNS, f"osmid/{id}").bytes)
        node_props.append(json.dumps(properties))

    location_arr = pa.array(
        [location] * len(node_ids), pa.dictionary(pa.int32(), pa.string())
    )
    node_ids_arr = pa.array(node_ids, pa.binary(16))
    node_props_arr = pa.array(node_props, pa.string())
    node_coords_arr = ga.as_geoarrow([str(point) for point in node_coords])

    return pa.Table.from_arrays(
        [location_arr, node_ids_arr, node_props_arr, node_coords_arr],
        ["location", "id", "properties", "geometry"],
    )


def process_edges(location: str, G: nx.MultiDiGraph):
    ureg = UnitRegistry()

    lines = []
    sources = []
    targets = []
    props = []

    for source, target, meta in list(G.edges(data=True))[:100]:
        properties = deepcopy(meta)

        # we internally store UUIDs for nodes and edges. so we create a static mapping
        # via UUID v5 ans store the original values in the properties.
        source_uuid = uuid5(NAMESPACE_DNS, f"osmid/{source}")
        target_uuid = uuid5(NAMESPACE_DNS, f"osmid/{target}")
        properties["osmid_source"] = source
        properties["osmid_target"] = target

        if "maxspeed" in properties:
            properties["maxspeed_m_s"] = (
                ureg(properties["maxspeed"]).to("m/s").magnitude
            )

        if "geometry" in properties:
            lines.append(properties["geometry"])
            del properties["geometry"]
        else:
            # when there is no geometry, we have a street with no additional
            # inner nodes. i.e. a straight line.
            source_tuple = G.nodes[source]["x"], G.nodes[source]["y"]
            target_tuple = G.nodes[target]["x"], G.nodes[target]["y"]
            lines.append(LineString([source_tuple, target_tuple]))

        props.append(json.dumps(properties))
        sources.append(source_uuid.bytes)
        targets.append(target_uuid.bytes)

    location_arr = pa.array(
        [location] * len(sources), pa.dictionary(pa.int32(), pa.string())
    )
    source_array = pa.array(sources, pa.binary(16))
    target_array = pa.array(targets, pa.binary(16))
    props_array = pa.array(props, pa.string())
    geo_lines = ga.as_geoarrow([str(line) for line in lines])

    return pa.Table.from_arrays(
        [location_arr, source_array, target_array, props_array, geo_lines],
        ["location", "source", "target", "properties", "geometry"],
    )

In [36]:
location_name = "london"
lng, lat = -0.13381370382489707, 51.518898098201326
resolution = 6

G = ox.graph_from_point(
    (lng, lat),
    dist=3000,
    network_type="all",
)
# Gp = ox.projection.project_graph(G)

In [ ]:
nodes = process_nodes("london", G)
pq.write_table(nodes, "nodes.parquet")

edges = process_edges("london", G)
pq.write_table(edges, "edges.parquet")

In [ ]:
def visualize_polygon(polyline, color, tiles="OpenStreetMap"):
    polyline.append(polyline[0])
    lat = [p[0] for p in polyline]
    lng = [p[1] for p in polyline]
    m = folium.Map(
        location=[sum(lat) / len(lat), sum(lng) / len(lng)], zoom_start=14, tiles=tiles
    )
    my_PolyLine = folium.PolyLine(locations=polyline, weight=4, color=color)
    m.add_child(my_PolyLine)
    return m


cell = h3.latlng_to_cell(lng, lat, resolution)
polygons = h3.cells_to_geo([cell])
polygon = list(polygons["coordinates"][0])

visualize_polygon(polygon, "red", "CartoDB dark_matter")